In [69]:
#PRACTICE USING AADIL'S ARTICLE - TO BE IMPLEMENTED INTO THE MINST DATABASE FOR IDENTIFYING NUMBERS
#Establishing number of nodes in an array
n = [2,3,3,1]

In [70]:
import numpy as np
#Neural Network Layer Weights. If l is the current indexed layer, then the dimensions of the weight matrix should be n^[l]xn^[(l-1)] where n[l]
#are the features in that layer
W1 = np.random.randn(n[1], n[0])
W2 = np.random.randn(n[2], n[1])
W3 = np.random.randn(n[3], n[2])
#This creates a 2 dimensional array

#Neural Network bias weights. Biases are just an n[l] x 1 matrix since each node only has 1 bias.
b1 = np.random.randn(n[1], 1)
b2 = np.random.randn(n[2], 1)
b3 = np.random.randn(n[3], 1)

In [71]:
#print the values as a test

print("Weights for layer 1 shape:", W1.shape)
print("Weights for layer 2 shape:", W2.shape)
print("Weights for layer 3 shape:", W3.shape)
print("bias for layer 1 shape:", b1.shape)
print("bias for layer 2 shape:", b2.shape)
print("bias for layer 3 shape:", b3.shape)

Weights for layer 1 shape: (3, 2)
Weights for layer 2 shape: (3, 3)
Weights for layer 3 shape: (1, 3)
bias for layer 1 shape: (3, 1)
bias for layer 2 shape: (3, 1)
bias for layer 3 shape: (1, 1)


In [72]:
#Neural Network Training Data - Input
X = np.array([
    [150, 70],
    [254, 73],
    [312, 68],
    [120, 60],
    [154, 61],
    [212, 65],
    [216, 67],
    [145, 67],
    [184, 64],
    [130, 69]
])
print(X.shape)

#Vectorization
A0 = X.T

#confirm transpose
print(A0.shape)

(10, 2)
(2, 10)


In [73]:
#Neural Network - Labels

y =  np.array([
    0,  
    1,   
    1, 
    0,
    0,
    1,
    1,
    0,
    1,
    0
])

m = 10

#Must be in the form n[3]^m since there is only 1 binary result of cardiovacular disease for each piece of training data
Y = y.reshape(n[3], m)
Y.shape

(1, 10)

In [74]:
#define the activation function
def sigmoid(arr):
    return 1 / (1 + np.exp(-1 * arr))

In [81]:
#create a forward propogation function
def forward_prop(W, b, A):
    """
    Rakshit's Notes:
    Note here that I add b which is a vector to a matrix. This is not mathematically defined but in Numpy we can "broadcast" the column values
     of b so that each entry of the b vector is repeatedly applied to the columns of the W @ A matrix
     
     You can read more by checking out this stack overflow question - https://stackoverflow.com/questions/15744402/numpy-matrix-plus-column-vector
     And the Numpy docs - https://numpy.org/doc/stable/user/basics.broadcasting.html
    """ 
    Z =  W @ A + b
    return sigmoid(Z)

In [76]:
A1 = forward_prop(W1, b1, A0)
#initial check for function working as intended
assert A1.shape == (n[1], m)

In [77]:
A2 = forward_prop(W2, b2, A1)
A3 = forward_prop(W3, b3, A2)
A3

array([[0.88636048, 0.88636048, 0.88636048, 0.88636048, 0.88636048,
        0.88636048, 0.88636048, 0.88636048, 0.88636048, 0.88636048]])

In [79]:
def cost(y_hat, y):
  """
  y_hat should be a n^L x m matrix
  y should be a n^L x m matrix
  """
  # 1. losses is a n^L x m
  """
  Rakshit's Notes:
  Bernoulli distribution based "binary cross-entropy loss" calculation. Essentially we use this since we use sigmoid (which outputs the probability of cardiovascular disease) as our 
  activation function and because the Neural Network returns a true or false answer, just like a bernoulli trial.
  
  Therefore, we can reduce the cost by maximizing the negative of the cost function. (ECE 204 application lol)
  
  Also note that this is the log of the intuitive Bernoulli PDF. This is done so that we only have to deal with a summation rather than a multiplication
  of the test samples. And, as you may know from calculus courses, 
  """
  losses = - ( (y * np.log(y_hat)) + (1 - y)*np.log(1 - y_hat) )

  m = y_hat.reshape(-1).shape[0]

  # 2. summing across axis = 1 means we sum across rows, 
  #   making this a n^L x 1 matrix
  summed_losses = (1 / m) * np.sum(losses, axis=1)

  # 3. unnecessary, but useful if working with more than one node
  #   in output layer
  return np.sum(summed_losses)